#**Temperature Downscaling of Amsterdam City using LightGBM Model**

**Author:** [Fatemeh Chajaei](https://github.com/FatemehCh97)

# Load Packages

In [ ]:
from lightgbm import LGBMRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import time
import os

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from scipy.stats import skew

# Data

In [ ]:

param_path = r"D:\Code\Amsterdam Parameter\Train_Test"

# df_train_avg = pd.read_csv(os.path.join(param_path, "Parameters_DataFrame.csv"))
# df_test_avg = pd.read_csv(os.path.join(param_path, "Test_Parameters_DataFrame.csv"))

# df_train_max = pd.read_csv(os.path.join(param_path, "Parameters_DataFrame_max.csv"))
# df_test_max = pd.read_csv(os.path.join(param_path, "Test_Parameters_DataFrame_max.csv"))

# df_train_min = pd.read_csv(os.path.join(param_path, "Parameters_DataFrame_min.csv"))
# df_test_min = pd.read_csv(os.path.join(param_path, "Test_Parameters_DataFrame_min.csv"))

"""Train/Test Avg"""
df = pd.read_csv(os.path.join(param_path, "Train_Parameters_DataFrame.csv"))
df_test = pd.read_csv(os.path.join(param_path, "Test_Parameters_DataFrame.csv"))

we do this for each selected days (January 17, 18, and 19, May 4, 5, and 6, July 18, 19, and 20, and October 24, 25, and 26, 2017)

In [ ]:
amst_param_path = r"D:\Code\Temp_Prediction_Amsterdam\Param\Parameters_DataFrame_AVG_May_04.csv"
amst_df = pd.read_csv(amst_param_path)

In [ ]:
X = df[df.columns[1:15]]
X_test = df_test[df_test.columns[1:15]]
X_amst = amst_df[amst_df.columns[1:15]]


df_all = np.concatenate((X, X_test, X_amst))

In [ ]:
scaler = MinMaxScaler()
Xs = scaler.fit_transform(df_all)


y = df.loc[:,['Temp']]
y = y.to_numpy()
y = np.ravel(y)
Xtrain = Xs[0:75268]


y_test = df_test.loc[:,['Temp']]
y_test = y_test.to_numpy()
y_test = np.ravel(y_test)
Xtest = Xs[75268:2374848]


Xamst = Xs[2374848:]

# HyperParameter Tuning

In [ ]:
cv = KFold(n_splits=10, random_state=1, shuffle=True)

param_hyperopt= {
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(1)),
    'n_estimators': scope.int(hp.quniform('n_estimators', 100, 2500, 50)),
    'max_depth': scope.int(hp.quniform('max_depth', 1, 50, 1)),
    'num_leaves': scope.int(hp.quniform('num_leaves', 1, 50, 1)),
    'reg_lambda': hp.uniform('reg_lambda', 0.01, 1.0),
    'boosting_type': hp.choice('boosting_type', ['gbdt', 'dart']),
    'subsample': hp.uniform('subsample', 0.01, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.01, 1.0),
    # 'reg_lambda': hp.uniform('log-uniform', 1e-9, 1),      # L2 regularization
    # 'reg_alpha': hp.uniform('log-uniform', 1e-9, 1),       # L1 regularization
    }

In [ ]:
def hyperopt(param_space, X_train, y_train, num_eval):

    start = time.time()

    def objective_function(params):
        reg = LGBMRegressor(**params)
        score = cross_val_score(reg, Xtrain, y, cv=5).mean()
        return {'loss': -score, 'status': STATUS_OK}

    trials = Trials()
    best_param = fmin(objective_function,
                      param_space,
                      algo=tpe.suggest,
                      max_evals=num_eval,
                      trials=trials,
                      rstate= np.random.default_rng(1))
    loss = [x['result']['loss'] for x in trials.trials]

    #best_param_values = [x for x in best_param.values()]

    if best_param['boosting_type'] == 0:
        boosting_type = 'gbdt'
    else:
        boosting_type= 'dart'



    reg_best = LGBMRegressor(learning_rate=best_param['learning_rate'],
                                  max_depth=int(best_param['max_depth']),
                                  n_estimators=int(best_param['n_estimators']),
                                  num_leaves=int(best_param['num_leaves']),
                                  boosting_type=boosting_type,
                                  colsample_bytree=best_param['colsample_bytree'],
                                  subsample=best_param['subsample'],
                                  reg_lambda=best_param['reg_lambda'])
    reg_best.fit(Xtrain, y)

    print("")
    print("##### Results")
    print("Score best parameters: ", min(loss)*-1)
    print("Best parameters: ", best_param)
    print("Time elapsed: ", time.time() - start)
    print("Parameter combinations evaluated: ", num_eval)

    return trials, reg_best

In [ ]:
results_hyperopt, reg_best = hyperopt(param_hyperopt, Xtrain, y, 20)

Best HP

In [ ]:
reg_best = LGBMRegressor(learning_rate=0.045,
                              max_depth=20,
                              n_estimators=3000,
                              num_leaves=28,
                              boosting_type='gbdt',
                              colsample_bytree=0.921596100038763,
                              subsample=0.21744055366203258,
                              reg_lambda=0.7329500731542772)

In [ ]:
reg_best.fit(Xtrain, y)

In [ ]:
df_test["y_pred"] = reg_best.predict(Xtest)
df["y_pred"] = reg_best.predict(Xtrain)

In [ ]:
amst_df["Temp_Pred"] = reg_best.predict(Xamst)

In [ ]:
prediction_outputpath = r"D:\Code\Temp_Prediction_Amsterdam\Preds\AVG"

amst_df_out = amst_df[['PointID', 'X', 'Y', 'Temp_Pred']].copy()
amst_df_out.to_csv(os.path.join(outputpath, 'AvgTemp_Prediction_Jan_19.csv'),index=False)


amst_df2.to_csv(os.path.join(outputpath, 'AvgTemp_Prediction_May_04_DataFrame.csv'),index=False)

# Evaluation

In [ ]:
RMSE_train = np.round(mean_squared_error(df["Temp"], df["y_pred"], squared=False),3)
MAE_train = np.round(mean_absolute_error(df["Temp"], df["y_pred"]),3)
RMSE_test = np.round(mean_squared_error(df_test["Temp"], df_test["y_pred"], squared=False),3)
MAE_test = np.round(mean_absolute_error(df_test["Temp"], df_test["y_pred"]),3)
print ("RMSE Train: ", RMSE_train, "      RMSE Test: ", RMSE_test)
print ("MAE Train:  ", MAE_train, "      MAE Test: ", MAE_test)

In [ ]:
MAPE = mean_absolute_percentage_error(df_test["Temp"], df_test["y_pred"])
print("MAPE: ", MAPE)

In [ ]:
from sklearn.metrics import r2_score
r2 = r2_score(df_test["Temp"], df_test["y_pred"])
print('r2 score for perfect model is', r2)

Error Disturbtion

In [ ]:
errors = df_test["Temp"]-df_test["y_pred"]

mean = np.mean(errors)
std_dev = np.std(errors)
median = np.median(errors)
skewness = skew(errors)


# Plot the histogram
plt.hist(errors, bins=30, edgecolor='black', alpha=0.5)

# Add add additional statistics values as text to histogram
plt.text(2.32,690000 , f'Mean: {mean:.3f}', ha='center', fontsize=12, fontname='Times New Roman')
plt.text(2.35, 610000 , f'Median: {median:.3f}', ha='center', fontsize=12, fontname='Times New Roman')
plt.text(2.385, 530000, f'Std Dev: {std_dev:.3f}', ha='center', fontsize=12, fontname='Times New Roman')
plt.text(2.49, 440000, f'Skewness: {skewness:.3f}', ha='center', fontsize=12, fontname='Times New Roman')

# Add labels and title
plt.xlabel('Error', fontsize=14, fontname='Times New Roman')
plt.ylabel('Frequency',labelpad=10, fontsize=14, fontname='Times New Roman')
plt.title('Error Distribution Histogram', fontsize=14, fontname='Times New Roman')

# Save plot
plt.savefig(r'D:\Code\Temp_Prediction_Amsterdam\Preds\AVG\Results\Error_Distribution.png', bbox_inches='tight', dpi=300)

# Show the plot
plt.show()

Regression Plot

In [ ]:
sns.regplot(x='Temp', y='y_pred', data=df_test,  line_kws={'color': 'slategrey', 'linewidth': 0.9},
            scatter_kws={'color': '#1eada5', 's': 5, 'facecolors': 'lightseagreen', 'linewidths': 0.5})

# Set plot title and labels
# plt.title('Regression Plot of Predicted vs Actual Temperature')
plt.xlabel("UrbClim Temperature (K)", labelpad=10, fontname='Times New Roman')
plt.ylabel("Predicted Temperature (K)", labelpad=10, fontname='Times New Roman')

# Remove the top and right spines
sns.despine(top=True, right=True)

# Display evaluation metrics on the plot
plt.text(295, 272, f'R2: {r2:.3f}', fontsize=12, ha='center', fontname='Times New Roman')
plt.text(295.5, 277, f'MAE: {MAE_test:.3f}', fontsize=12, ha='center', fontname='Times New Roman')
plt.text(295.8, 282, f'RMSE: {RMSE_test:.3f}', fontsize=12, ha='center', fontname='Times New Roman')

# Save the plot
plt.savefig(r'D:\Code\Temp_Prediction_Amsterdam\Preds\AVG\Results\RegressionPlot_8.png', bbox_inches='tight', dpi=300)

# Show the plot
plt.show()


Feature Importance

In [ ]:
def plotImp(model, X , num = 20, fig_size = (40, 20)):
    feature_imp = pd.DataFrame({'Value':(model.feature_importances_/sum(model.feature_importances_))*100,'Feature':X.columns})
    plt.figure(figsize=fig_size)
    sns.set(font_scale = 5)
    sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value",
                                                        ascending=False)[0:num])


    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig(r'D:\Code\Temp_Prediction_Amsterdam\Preds\AVG\Results\lgbm_FeatureImportances.png', bbox_inches='tight', dpi=300)
    plt.show()


plotImp(reg_best, X_test)

Correlation Plot

In [ ]:
plt.figure(figsize=(27, 16))

# Plot the correlation between Independent Variable and Temperature as a scatter plot
sns.scatterplot(data=amst_df, x='H', y='Temp_Pred', color='lightseagreen', edgecolor='lightseagreen', s=6, linewidths=1.5)

# Set plot title and labels
plt.title('Correlation Plot: Temperature vs. Area', fontname='Times New Roman')
plt.xlabel('Area', fontname='Times New Roman')
plt.ylabel('Temperature', fontname='Times New Roman')

# Remove the top and right spines
sns.despine(top=True, right=True)

# Save the plot
plt.savefig(r'D:\Code\Temp_Prediction_Amsterdam\Preds\AVG\Results\CorrPlot_H_1.png', bbox_inches='tight', dpi=300)

# Show the plot
plt.show()